In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

###### ToTensor로 바꿔야 입력이 됨  

trans = transforms.Compose([  
----transforms.ToTensor()   
])

###### train_data를 imageFolder를 통해서 dataset으로 사용  

train_data = torchvision.datasets.ImageFolder(root='./custom_data/train_data', transform=trans)


data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers=2)

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(  # Sequential로 묶어서 FC 두번 진행
            nn.Linear(16*13*29, 120),
            nn.ReLU(),
            nn.Linear(120,2)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        #print(out.shape) -> 각 layer를 통과하고 나와서 어떤 크기로 되는지 바로 보임
        out = self.layer2(out)
        #print(out.shape) # 확인 후엔 꼭 지워줘야함
        out = out.view(out.shape[0], -1)
        #print(out.shape)
        out = self.layer3(out)
        return out

In [6]:
#testing 
net = CNN().to(device)
test_input = (torch.Tensor(3,3,64,128)).to(device)
test_out = net(test_input)

#결과 :
##torch.Size([3,6,30,62])
##torch.Size([3,16,13,29])
##torch.Size([3,6032])

In [11]:
optimizer = optim.Adam(net.parameters(), lr=0.00005)
loss_func = nn.CrossEntropyLoss().to(device) #loss는 cross_entropy_loss 사용

total_batch = len(data_loader)

epochs = 7
for epoch in range(epochs):  
----avg_cost = 0.0  
----for num, data in enumerate(data_loader):  
--------imgs, labels = data  
--------imgs = imgs.to(device)  
--------labels = labels.to(device)  
--------optimizer.zero_grad()  
--------out = net(imgs)  
--------loss = loss_func(out, labels)  
--------loss.backward()  
--------optimizer.step()
        
--------avg_cost += loss / total_batch
        
----print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')

###### 모델을 save해서 다시 불러오는 방법
torch.szve()를 이용해서 그 network의 state_dict값 저장
  
torch.save(net.state_dict(), "./model/model.pth")

In [12]:
#불러오려면 network의 모형은 같아야함
new_net = CNN().to(device) # net과 일치함

new_net.load_state_dict(torch.load('./model/model.pth'))

###### new_net과 net 비교
print(net.layer1[0])
print(new_net.layer1[0])

##net.layer1[0](Convolution layer)의 weight값의 3번째 index 뽑기 
print(net.layer1[0].weight[0][0][0])  
print(new_net.layer1[0].weight[0][0][0])

net.layer1[0].weight[0] == new_net.layer1[0].weight[0]  
  
##결과는 둘이 똑같이 나옴

###### 학습시킨 model test
trans=torchvision.transforms.Compose([  
----transforms.Resize((64,128)),  ##resize  
----transforms.ToTensor()  ##ToTensor  
])  
test_data = torchvision.datasets.ImageFolder(root='./custom_data/test_data', transform=trans)

test_set = DataLoader(dataset = test_data, batch_size = len(test_data))  
#그냥 한 번에 쓸 수 있게 test dataset의 길이 만큼으로 그냥 batch_size를 한 번에 받을 수 있게

with torch.no_grad():
----for num, data in enumerate(test_set):
-------imgs, label = data
-------imgs = imgs.to(device)
-------label = label.to(device)
        
-------prediction = net(imgs)
        
-------correct_prediction = torch.argmax(prediction, 1) == label
        
-------accuracy = correct_prediction.float().mean()
-------print('Accuracy:', accuracy.item())

### 정확도 0.579정도 나옴 => 올리려면 epoch를 늘리거나 lr을 키움